In [1]:
import bifacial_radiance
import os
import pandas as pd

In [2]:
inifile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\ini_highAzimuth.ini'
#inifile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\ini_gencumsky.ini'
#inifile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\ini_1axis.ini'

weatherfile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\724666TYA.csv'
testfolder = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Modelchain'

In [3]:
(Params)= bifacial_radiance.load.readconfigurationinputfile(inifile=inifile)

Loading times to restrict weather data to


In [4]:
Params

({'weatherFile': '724666TYA.CSV',
  'getEPW': False,
  'simulationname': '_test_high_azimuth_angle_modelchain',
  'moduletype': 'test-module',
  'rewriteModule': True,
  'cellLevelModule': False,
  'axisofrotationTorqueTube': False,
  'torqueTube': False,
  'hpc': False,
  'tracking': False,
  'cumulativeSky': False,
  'selectTimes': True,
  'latitude': '37.5',
  'longitude': '-77.6'},
 {'albedo': 'white_EPDM',
  'nMods': 10,
  'nRows': 3,
  'gcrorpitch': 'pitch',
  'pitch': 1.5,
  'azimuth': 30.0,
  'clearance_height': 0.2,
  'tilt': 10.0},
 {'starttime': '06_17_13', 'endtime': '06_17_13'},
 {'bifi': 0.8,
  'numpanels': 1,
  'xgap': 0.0,
  'ygap': 0.0,
  'zgap': 0.0,
  'x': 1.59,
  'y': 0.95},
 None,
 None,
 {'sensorsy': (8, 9), 'modWanted': 5, 'rowWanted': 2},
 None,
 {'alpha_sc': 0.0039216,
  'a_ref': 1.80227,
  'I_L_ref': 10.5308,
  'I_o_ref': 2.28e-11,
  'R_sh_ref': 900.982,
  'R_s': 0.285529,
  'Adjust': 4.39011})

In [5]:
Params[0]['testfolder'] = testfolder
Params[0]['weatherFile'] = weatherfile
Params[2].update({'starttime': '06_17_13', 'endtime':'06_17_14'}); 

In [15]:
Params[6]['modWanted'] = [1, 3]

In [16]:
demo2, analysis = bifacial_radiance.modelchain.runModelChain(*Params ) 


New bifacial_radiance simulation starting. 
Version:  0.4.1+14.g67db8ee.dirty
path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Modelchain
Loading times to restrict weather data to
analysisParamsDict['modWanted'] set to middle module by default
Reading weather file C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\724666TYA.csv
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.800 avg
1 nonzero albedo values.
Available module names: ['PrismSolar-Bi60', 'basic-module', 'test-module']

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwri

c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\mismatch.py:220: FutureWarning: outer method for ufunc <ufunc 'subtract'> is not implemented on pandas objects. Returning an ndarray, but in the future this will raise a 'NotImplementedError'. Consider explicitly converting the DataFrame to an array with '.to_numpy()' first.
  return (np.abs(np.subtract.outer(data,data)).sum()/float(data.__len__())**2 / np.mean(data))*100


In [8]:
demo2.trackerdict

{'2021-06-17_1300': {'surf_azm': 30.0,
  'surf_tilt': 10.0,
  'theta': 10.0,
  'ghi': 959,
  'dhi': 172,
  'temp_air': 28.9,
  'wind_speed': 6.2,
  'skyfile': 'skies\\sky2_39.742_-105.179_2021-06-17_1300.rad',
  'clearance_height': 0.2,
  'radfile': 'objects\\1axis2021-06-17_1300__C_0.20000_rtr_1.50000_tilt_10.00000_10modsx3rows_origin0,0.rad',
  'scene': {'module': {'x': 1.59, 'y': 0.95, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.59, 'sceney': 0.95, 'scenez': 0.0, 'numpanels': 1, 'bifi': 0.8, 'text': '! genbox black test-module 1.59 0.95 0.02 | xform -t -0.795 -0.475 0 -a 1 -t 0 0.95 0', 'modulefile': 'objects\\test-module.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.0, 'ygap': 0.0, 'zgap': 0.0}, 'modulefile': 'objects\\test-module.rad', 'hpc': False, 'gcr': 0.6333333333333333, 'text': '!xform -rx 10.0 -t 0 0 0.2824828843917919 -a 10 -t 1.59 0 0 -a 3 -t 0 1.5 0 -i 1 -t -6.36 -1.5 0 -rz 150.0 -t 0 0 0 objects\\test-module.rad', 'radfiles': 'objects\\1axis2021-06-17_1300_

In [9]:
trackerdict = demo2.trackerdict
keys = list(demo2.trackerdict.keys())

In [10]:
frontirrad = trackerdict[keys[0]]['AnalysisObj'].Wm2Front

KeyError: 'AnalysisObj'

In [ ]:
type(frontirrad)

In [ ]:
#CEC Module
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.
modfilter2 = db.index.str.startswith('Pr') & db.index.str.endswith('BHC72-400')
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

In [ ]:
type(CECMod)

In [ ]:
CECModParamsDict = Params[-1]

In [ ]:
CECModParamsDict

In [ ]:
pd.DataFrame(CECModParamsDict, index=[0])

In [ ]:
import numpy as np

In [ ]:
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[1] = np.nan
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[2] = np.nan
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[3] = np.nan
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[4] = np.nan
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[5] = np.nan
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[6] = np.nan
demo2.trackerdict[keys[0]]['AnalysisObj'].Wm2Front[7] = np.nan

In [ ]:
type(CECMod)

In [ ]:
CECMod)

In [ ]:
trackerdict = demo2.calculateResults(CECMod = CECMod)

In [ ]:
CECMod.Adjust

In [ ]:
            print("Error: Mising/wrong parameters for CECMod, setting dictionary to None.",
                  "MAke sure to include alpha_sc, a_ref, I_L_ref, I_o_ref, ",
                  "R_sh_ref, R_s, and Adjust"
                  "Performance calculations, if performed, will use default module")

In [ ]:
demo2.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)


In [ ]:
pd.read_csv(os.path.join('results','Final_Results.csv'))